In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from scipy.signal import correlate
import concurrent.futures
import csv
from tqdm import tqdm

# Define sequence length table
sequence_length_table = {
    '2': 200, '3': 400, '4': 400, '5': 200, '5i': 200, '6': 200, '6i': 200,
    '7': 100, '7i': 100, '8': 800, '8i': 800, '9': 800, '9i': 800, '10': 1600, '10i': 1600,
    '11': 3200, '11i': 3200, '12': 400, '12i': 400, '13': 200, '13i': 200, '14': 100, '14i': 100,
    '15': 800, '15i': 800
}

# Define file paths
data_files = {
    "Small trajectory test": 'own/data/4.csv',
    "Real traffic test": 'own/data/6.csv',
    "Zero-shot large trajectory test": 'own/data/7.csv'
}

data_int_files = {
    "Small trajectory test": 'own/data-int/4.csv',
    "Real traffic test": 'own/data-int/6.csv',
    "Zero-shot large trajectory test": 'own/data-int/7.csv'
}

# Function to load and preprocess a single file
def load_and_preprocess_file(file, start_offset, end_offset):
    df = pd.read_csv(file)
    df['datetime'] = pd.to_datetime(df['date/time'], errors='coerce')
    start_time = df['datetime'].min() + pd.Timedelta(minutes=start_offset)
    end_time = df['datetime'].max() - pd.Timedelta(minutes=end_offset)
    return df[(df['datetime'] >= start_time) & (df['datetime'] <= end_time)]

# Function to load and preprocess all files
def load_and_preprocess_all_files():
    all_data = {}
    for key, file in tqdm(data_files.items(), desc="Loading and preprocessing data files"):
        if "4.csv" in file:
            start_offset = 1.5
            end_offset = 1.5
        elif "6.csv" in file:
            start_offset = 2
            end_offset = 1
        else:
            start_offset = 1
            end_offset = 1

        df = load_and_preprocess_file(file, start_offset, end_offset)
        all_data[key] = df

    for key, file in tqdm(data_int_files.items(), desc="Loading and preprocessing interpolated data files"):
        if "4.csv" in file:
            start_offset = 1.5
            end_offset = 1.5
        elif "6.csv" in file:
            start_offset = 2
            end_offset = 1
        else:
            start_offset = 1
            end_offset = 1

        df = load_and_preprocess_file(file, start_offset, end_offset)
        all_data[key + " (interpolated)"] = df

    return all_data

# Load and preprocess all data
print("Loading and preprocessing all data...")
all_data = load_and_preprocess_all_files()
print("Data loading and preprocessing complete.")

2024-06-17 14:55:45.891312: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-17 14:55:47.071988: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-06-17 14:55:47.072077: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-06-

Loading and preprocessing all data...


Loading and preprocessing data files: 100%|██████████| 3/3 [00:01<00:00,  1.62it/s]
Loading and preprocessing interpolated data files: 100%|██████████| 3/3 [00:01<00:00,  1.60it/s]

Data loading and preprocessing complete.


In [2]:
# Standardize features
features = ['gyro_x', 'gyro_y', 'gyro_z', 'accel_x', 'accel_y', 'accel_z', 'x_cam', 'y_cam', 'z_cam']
targets = ['x', 'y']

scalers = {}
for key in tqdm(all_data, desc="Standardizing features"):
    scaler = StandardScaler()
    all_data[key][features] = scaler.fit_transform(all_data[key][features])
    scalers[key] = scaler

print("Feature standardization complete.")

Standardizing features: 100%|██████████| 6/6 [00:00<00:00, 65.20it/s]

Feature standardization complete.


In [ ]:
# Function to get sequence length from the model name
def get_sequence_length_from_model_name(model_name):
    base_name = os.path.basename(model_name).replace('trajectory_model', '').replace('.h5', '')
    return sequence_length_table.get(base_name, None)

# Function to create sequences for each dataset
def create_sequences(data, sequence_length):
    sequences = []
    labels = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length][features].values)
        labels.append(data.iloc[i+sequence_length][targets].values)
    return np.array(sequences), np.array(labels)

# Function to create sequences with multithreading
def create_sequences_for_key(key, seq_len):
    sequences, labels = create_sequences(all_data[key], seq_len)
    return key, sequences, labels

# Store sequences and labels in a dictionary
sequences_labels = {}

# Find all .h5 model files in the root folder
def find_model_files(root_folder):
    model_files = []
    for file in os.listdir(root_folder):
        if file.endswith(".h5") and "checkpoint" not in file:
            model_files.append(os.path.join(root_folder, file))
    return model_files

# Find all model files
print("Searching for model files...")
root_folder = "."
model_files = find_model_files(root_folder)
print(f"Found {len(model_files)} model files.")

# Filter out models not in sequence_length_table
model_files = [mf for mf in model_files if get_sequence_length_from_model_name(mf) is not None]

# Group datasets by sequence length
datasets_by_seq_len = {}
for model_file in model_files:
    model_name = os.path.basename(model_file)
    seq_len = get_sequence_length_from_model_name(model_name)
    if seq_len not in datasets_by_seq_len:
        datasets_by_seq_len[seq_len] = []
    
    if 'i' in model_name:
        datasets_by_seq_len[seq_len].extend([key + " (interpolated)" for key in data_files.keys()])
    else:
        datasets_by_seq_len[seq_len].extend(data_files.keys())

# Create sequences using multithreading
for seq_len, keys in tqdm(datasets_by_seq_len.items(), desc="Creating sequences by sequence length"):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {executor.submit(create_sequences_for_key, key, seq_len): key for key in keys}
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc=f"Sequence length {seq_len}"):
            key, sequences, labels = future.result()
            sequences_labels[key] = (sequences, labels)

print("Sequence creation complete.")

Searching for model files...
Found 20 model files.


Sequence length 200:  17%|█▋        | 3/18 [1:17:30<4:21:30, 1046.02s/it]

In [ ]:
# Function to find all .h5 model files
def find_model_files(root_folder):
    model_files = []
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            if file.endswith(".h5") and "checkpoint" not in file:
                model_files.append(os.path.join(root, file))
    return model_files

# Find all model files
print("Searching for model files...")
root_folder = "."
model_files = find_model_files(root_folder)
print(f"Found {len(model_files)} model files.")

In [ ]:
# Function to calculate RMSE
def calculate_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Function to apply a moving average filter to the predicted data
def moving_average(data, window_size):
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')

# Function to process data after filtering, scaling, shifting, and lag correction
def process_data(y_true, y_pred, window_size):
    print(f"Processing data with window size {window_size}...")
    
    # Apply moving average filter to predicted data
    y_pred_filtered = np.array([moving_average(y_pred[:, i], window_size) for i in range(y_pred.shape[1])]).T

    # Align the test data to match the length of the filtered predictions
    y_test_aligned = y_true[len(y_true) - len(y_pred_filtered):]

    # Calculate scaling factors using standard deviation
    scaling_factors = np.std(y_test_aligned, axis=0) / np.std(y_pred_filtered, axis=0)

    # Apply scaling factors to the filtered predicted data
    y_pred_scaled = y_pred_filtered * scaling_factors

    # Calculate the mean of the aligned ground truth and scaled predictions
    mean_y_test = np.mean(y_test_aligned, axis=0)
    mean_y_pred = np.mean(y_pred_scaled, axis=0)

    # Calculate the shift required to align the means
    shift = mean_y_test - mean_y_pred

    # Apply the shift to the scaled predicted data
    y_pred_scaled_shifted = y_pred_scaled + shift

    # Determine the lag that maximizes the cross-correlation between the ground truth and the predicted data
    lag_x = find_best_shift(y_test_aligned[:, 0], y_pred_scaled_shifted[:, 0])
    lag_y = find_best_shift(y_test_aligned[:, 1], y_pred_scaled_shifted[:, 1])

    # Create copies of the data to apply the shifts
    y_pred_final_x = np.roll(y_pred_scaled_shifted[:, 0], shift=lag_x)
    y_pred_final_y = np.roll(y_pred_scaled_shifted[:, 1], shift=lag_y)

    # Combine the shifted dimensions back into a single array
    y_pred_final = np.column_stack((y_pred_final_x, y_pred_final_y))

    return y_test_aligned, y_pred_final, scaling_factors, shift, lag_x, lag_y

# Function to evaluate a model on a dataset
def evaluate_model(model_path, dataset_key, window_sizes):
    print(f"Evaluating model {model_path} on {dataset_key}...")
    model = load_model(model_path)
    X, y = sequences_labels[dataset_key]
    y_pred = model.predict(X)
    
    results = []
    
    for window_size in window_sizes:
        print(f"Evaluating with window size {window_size}...")
        
        # RMSE without any filtering
        rmse_no_filter = calculate_rmse(y, y_pred)
        
        # RMSE after filtering
        y_true_filtered, y_pred_filtered, _, _, _, _ = process_data(y, y_pred, window_size)
        rmse_filter = calculate_rmse(y_true_filtered, y_pred_filtered)
        
        # RMSE after scaling, filtering, and shifting
        y_true_final, y_pred_final, scaling_factors, shift, lag_x, lag_y = process_data(y, y_pred, window_size)
        rmse_final = calculate_rmse(y_true_final, y_pred_final)
        
        # Error between expected end point and predicted end point
        expected_end_point = y_true_final[-1]
        predicted_end_point = y_pred_final[-1]
        end_point_error = np.sqrt((expected_end_point[0] - predicted_end_point[0])**2 + (expected_end_point[1] - predicted_end_point[1])**2)

        # Create plots
        plot_trajectories(y, y_pred, f"{dataset_key} ({os.path.basename(model_path)}, window size={window_size})", save_path=f"images/{dataset_key}_{os.path.basename(model_path)}_ws{window_size}.png")
        
        result = {
            "model": os.path.basename(model_path),
            "test": dataset_key,
            "window_size": window_size,
            "rmse_no_filter": rmse_no_filter,
            "rmse_filter": rmse_filter,
            "rmse_final": rmse_final,
            "end_point_error": end_point_error
        }
        
        results.append(result)
    
    return results

# Function to find the best shift
def find_best_shift(y_true, y_pred):
    correlation = correlate(y_true, y_pred)
    lag = correlation.argmax() - (len(y_pred) - 1)
    return lag

# Function to plot trajectories
def plot_trajectories(true_data, pred_data, title_suffix, save_path=None):
    plt.figure(figsize=(15, 5))
    plt.plot(true_data[:, 0], label=f'True X ({title_suffix})')
    plt.plot(pred_data[:, 0], label=f'Predicted X ({title_suffix})')
    plt.legend()
    plt.title(f'X Trajectory ({title_suffix})')
    if save_path:
        plt.savefig(save_path.replace('.png', '_x.png'))
    else:
        plt.show()

    plt.figure(figsize=(15, 5))
    plt.plot(true_data[:, 1], label=f'True Y ({title_suffix})')
    plt.plot(pred_data[:, 1], label=f'Predicted Y ({title_suffix})')
    plt.legend()
    plt.title(f'Y Trajectory ({title_suffix})')
    if save_path:
        plt.savefig(save_path.replace('.png', '_y.png'))
    else:
        plt.show()

    plt.figure(figsize=(10, 10))
    plt.plot(true_data[:, 0], true_data[:, 1], label=f'True Trajectory ({title_suffix})', color='blue')
    plt.plot(pred_data[:, 0], pred_data[:, 1], label=f'Processed Predicted Trajectory ({title_suffix})', color='red', linestyle='dashed')
    plt.legend()
    plt.xlabel('X Position')
    plt.ylabel('Y Position')
    plt.title(f'Trajectory Comparison ({title_suffix})')
    plt.axis('equal')
    plt.grid(True)
    if save_path:
        plt.savefig(save_path.replace('.png', '_xy.png'))
    else:
        plt.show()

# Define window sizes
window_sizes = [200, 400, 800, 1600, 2500, 3200]

# Function to run evaluations
def run_evaluations():
    results = []
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for model_file in model_files:
            model_name = os.path.basename(model_file)
            if 'i' in model_name:
                for test in data_int_files.keys():
                    futures.append(executor.submit(evaluate_model, model_file, test + " (interpolated)", window_sizes))
            else:
                for test in data_files.keys():
                    futures.append(executor.submit(evaluate_model, model_file, test, window_sizes))
        
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="Evaluating models"):
            try:
                results.extend(future.result())
            except Exception as e:
                print(f"Error: {e}")
    
    return results

# Run evaluations
print("Running evaluations...")
results = run_evaluations()
print("Evaluations complete.")

# Write results to CSV
print("Writing results to evaluation.csv...")
with open('evaluation.csv', 'w', newline='') as csvfile:
    fieldnames = ['model', 'test', 'window_size', 'rmse_no_filter', 'rmse_filter', 'rmse_final', 'end_point_error']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for result in tqdm(results, desc="Writing results to CSV"):
        writer.writerow(result)
print("Results written to evaluation.csv.")